In [18]:
import pandas
import os
from datetime import datetime

current_path = os.path.abspath('')
df1 = pandas.read_excel(current_path + '/../data/集思录.xlsx')
df2 = pandas.read_excel(current_path + '/../data/上市发行.xlsx')
df3 = pandas.read_excel(current_path + '/../data/发行结果.xlsx')
df4 = pandas.read_excel(current_path + '/../data/上市表现.xlsx')
df5 = pandas.read_excel(current_path + '/../data/正股价格.xlsx', converters={'股票代码':str})
df6 = pandas.read_excel(current_path + '/../data/正股信息.xlsx', converters={'A股代码':str})

df1['可转债代码'] = df1['证券代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x[1:-1].replace('\'','').split(',')][1])
df2['可转债代码'] = df2['转债代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df3['可转债代码'] = df3['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df4['可转债代码'] = df4['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])

df = pandas.merge(df1,df2, on='可转债代码', how='left')
df = pandas.merge(df,df3, on='可转债代码', how='left')
df = pandas.merge(df,df4, on='可转债代码', how='left')

for idx, row in df.iterrows():
    # 第一个是正股代码， 第二个是可转债代码
    symbol = [c.strip() for c in row['证券代码'][1:-1].replace('\'','').split(',')]
    name = [c.strip() for c in row['证券名称'][1:-1].replace('\'','').split(',')]
    # 可转债上市日期
    listing_date = row['上市公告日期']
    # 可转债配售股权登记日 (T-1日)
    registration = row['股权登记日']
    #print(symbol, name, listing_date, registration)
    # 网上发行日 (T日)
    day_t = row['网上发行日期_x'].strftime("%Y-%m-%d")
    # 发行公告日 (T-2日)，不一定准确，可能提前T-2日道之查询股价产生空数据
    day_t_2 = row['发行公告日'].strftime("%Y-%m-%d")

    #print(symbol, name, day_t_2, registration, day_t, listing_date)
    idx_t = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==day_t)].index[0]
    #print(price)

    df.loc[df['可转债代码']==symbol[1], 'T日开盘'] = df5.at[idx_t,'开盘']
    df.loc[df['可转债代码']==symbol[1], 'T日收盘'] = df5.at[idx_t,'收盘']
    df.loc[df['可转债代码']==symbol[1], 'T日均价'] = df5.at[idx_t,'均价']
    df.loc[df['可转债代码']==symbol[1], 'T日涨跌幅'] = df5.at[idx_t,'涨跌幅']
    
    df.loc[df['可转债代码']==symbol[1], 'T-1日开盘'] = df5.at[idx_t-1,'开盘']
    df.loc[df['可转债代码']==symbol[1], 'T-1日收盘'] = df5.at[idx_t-1,'收盘']
    df.loc[df['可转债代码']==symbol[1], 'T-1日均价'] = df5.at[idx_t-1,'均价']
    df.loc[df['可转债代码']==symbol[1], 'T-1日涨跌幅'] = df5.at[idx_t-1,'涨跌幅']
    
    df.loc[df['可转债代码']==symbol[1], 'T-2日开盘'] = df5.at[idx_t-2,'开盘']
    df.loc[df['可转债代码']==symbol[1], 'T-2日收盘'] = df5.at[idx_t-2,'收盘']
    df.loc[df['可转债代码']==symbol[1], 'T-2日均价'] = df5.at[idx_t-2,'均价']
    df.loc[df['可转债代码']==symbol[1], 'T-2日涨跌幅'] = df5.at[idx_t-2,'涨跌幅']
    
    df.loc[df['可转债代码']==symbol[1], 'T-3日开盘'] = df5.at[idx_t-3,'开盘']
    df.loc[df['可转债代码']==symbol[1], 'T-3日收盘'] = df5.at[idx_t-3,'收盘']
    df.loc[df['可转债代码']==symbol[1], 'T-3日均价'] = df5.at[idx_t-3,'均价']
    df.loc[df['可转债代码']==symbol[1], 'T-3日涨跌幅'] = df5.at[idx_t-3,'涨跌幅']
    
    idx_listing_date = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==listing_date)].index[0]
    
    df.loc[df['可转债代码']==symbol[1], 'T-2日'] = df5.at[idx_t-2,'日期']
    df.loc[df['可转债代码']==symbol[1], '可转债上市日正股收盘价'] = df5.at[idx_listing_date,'收盘']
    
    info = df6[df6['A股代码']==symbol[0]].index.tolist()
    if not info:
        continue
    df.loc[df['可转债代码']==symbol[1], 'A股总股本'] = df6[df6['A股代码']==symbol[0]][0, 'A股总股本']
    df.loc[df['可转债代码']==symbol[1], 'A股流通股本'] = df6[df6['A股代码']==symbol[0]][0, 'A股流通股本']

    
#df.to_excel(current_path + '/../data/data.xlsx', index=False)


003043
[1505]
80,000,000
603176
[]


IndexError: list index out of range